In [1]:
import numpy as np
import pandas as pd
import os
import csv


In [2]:
# define csv file
csv_file = '../Dataset/dataset_augmented.csv'
df = pd.DataFrame(columns=('Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11', 'Pos12', 'Pos13', 'Pos14', 'Pos15', 'Pos16', 'R', 'lambd'))

In [5]:
# Read logfiles

dir_path = '../Todos_6Helicenos'
logfiles = []
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        logfiles.append(path)

y = np.zeros(2)
i_data = 0
for logfile in logfiles:

    infile = open(os.path.join(dir_path,logfile),'r')

    # first, full of hydrogens (id 0)
    x = [0]*16

    ### Use logfile name to define the feature vector
    # Remove rubish from the logfile name
    logfile = logfile.replace('6Heliceno_','')
    logfile = logfile.replace('.log','')
    #print(logfile)
    # Split substituent elements 
    elements = logfile.split('_')
    #print(elements)
    for element in elements:
        xaux = element.split('-')
        i = int(xaux[0])-1
        halogen = xaux[1]
        if(halogen=='fluor'):
            halogen_id = 1
        elif(halogen=='cloro'):
            halogen_id = 2
        elif(halogen=='bromo'):
            halogen_id = 3
        elif(halogen=='yodo'):
            halogen_id = 4

        x[i] = halogen_id
    #print(x)

    ### Define targets from higher Rotatory Strength (R) and wave length
    # read all lines of logfile into a list
    lines = infile.readlines()

    # Find R data
    for line in lines:
        # check if string is present on the current line
        word = 'R(length)'
        if line.find(word) != -1:
            j_init = lines.index(line) + 1
            break

    # compute maximum R
    R_max = 0
    for j in range(100):
        line = lines[j+j_init]
        # read R
        columns = line.split()
        state = int(columns[0])
        R = float(columns[4])
        if(R>R_max): 
            R_max = R
            state_max = state
    #print(R_max)
    y[0] = R_max

    # Find wave length data
    for line in lines:
        words = 'oscillator strengths:'
        if line.find(words) != -1:
            j_init = lines.index(line) + 2
            break

    # Find wave length of the state with maximum R
    for line in lines[j_init:]:
        columns = line.split()
        #print(columns)
        if(len(columns)>0):
            word1 = columns[0]
            if(word1=='Excited'):
                state = columns[2]
                state = state.replace(':','')
                state = int(state)
                if(state==state_max):
                    #print(state)
                    lambd = float(columns[6])
                    break                
    #print(lambd)
    y[1] = lambd

    #print(y)

    ### Save data into a dataframe
    datum = []
    for i in range(16):
        datum.append(str(x[i]))
    datum.append(str(y[0]))
    datum.append(str(y[1]))
    print(datum)
    df.loc[i_data] = datum
    i_data = i_data+1

    ### also save the symmetric molecule
    x.reverse()
    datum_reverse = []
    for i in range(16):
        datum_reverse.append(str(x[i]))
    datum_reverse.append(str(y[0]))
    datum_reverse.append(str(y[1]))
    print(datum_reverse)
    df.loc[i_data] = datum_reverse
    i_data = i_data+1

['0', '0', '0', '0', '0', '0', '0', '2', '1', '0', '0', '0', '0', '0', '3', '3', '861.9503', '289.48']
['3', '3', '0', '0', '0', '0', '0', '1', '2', '0', '0', '0', '0', '0', '0', '0', '861.9503', '289.48']
['0', '1', '0', '0', '4', '0', '0', '0', '0', '4', '0', '3', '0', '0', '0', '0', '646.2645', '286.47']
['0', '0', '0', '0', '3', '0', '4', '0', '0', '0', '0', '4', '0', '0', '1', '0', '646.2645', '286.47']
['0', '2', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '986.9448', '277.39']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '2', '0', '986.9448', '277.39']
['0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '0', '916.7423', '278.03']
['0', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '916.7423', '278.03']
['0', '0', '3', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '962.5692', '277.25']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '3', '0

In [6]:
# Save dataframe into a csv file 
df.to_csv(csv_file)
df.shape

(152, 18)